In [1]:
from tensorflow.keras.utils import to_categorical
from keras.datasets import cifar10
from keras.applications import InceptionV3
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.layers import Lambda
from tensorflow.keras import layers, models

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [3]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

In [4]:
inputs = tf.keras.Input(shape=(32, 32, 3))
x = Lambda(lambda image: tf.image.resize(image, (224, 224)))(inputs)
x = tf.keras.applications.inception_v3.preprocess_input(x)
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
outputs = Dense(10, activation='softmax')(x)
model = Model(inputs, outputs)

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
early_stopping = EarlyStopping(
    monitor='accuracy',
    patience=1,
    restore_best_weights=True
)
model.fit(x = X_train, y= y_train, epochs=10, callbacks=[early_stopping])

Epoch 1/10
1563/1563 [==============================] - 1492s 951ms/step - loss: 0.5677 - accuracy: 0.8147
Epoch 2/10
1563/1563 [==============================] - 1585s 1s/step - loss: 0.4617 - accuracy: 0.8492
Epoch 3/10
1563/1563 [==============================] - 1473s 943ms/step - loss: 0.4469 - accuracy: 0.8549
Epoch 4/10
1563/1563 [==============================] - 1405s 899ms/step - loss: 0.4438 - accuracy: 0.8554
Epoch 5/10
1563/1563 [==============================] - 1395s 892ms/step - loss: 0.4389 - accuracy: 0.8585
Epoch 6/10
1563/1563 [==============================] - 1384s 885ms/step - loss: 0.4398 - accuracy: 0.8594
Epoch 7/10
1563/1563 [==============================] - 1382s 884ms/step - loss: 0.4399 - accuracy: 0.8584


In [7]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

313/313 - 272s - loss: 0.4221 - accuracy: 0.8697 - 272s/epoch - 867ms/step

Test accuracy: 0.869700014591217


In [8]:
model.save('NN/inception_cifar10.h5')